In [1]:
# The code was removed by Watson Studio for sharing.

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20191001123148-0000
KERNEL_ID = fb1deef0-f6d7-4cfa-a1e9-54a3df6550bc


# IEEE CIS Fraud Detection - ETL

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from pixiedust.display import *

Pixiedust database opened successfully
Table VERSION_TRACKER created successfully
Table METRICS_TRACKER created successfully

Share anonymous install statistics? (opt-out instructions)

PixieDust will record metadata on its environment the next time the package is installed or updated. The data is anonymized and aggregated to help plan for future releases, and records only the following values:

{
   "data_sent": currentDate,
   "runtime": "python",
   "application_version": currentPixiedustVersion,
   "space_id": nonIdentifyingUniqueId,
   "config": {
       "repository_id": "https://github.com/ibm-watson-data-lab/pixiedust",
       "target_runtimes": ["Data Science Experience"],
       "event_id": "web",
       "event_organizer": "dev-journeys"
   }
}
You can opt out by calling pixiedust.optOut() in a new cell.


Pixiedust runtime updated. Please restart kernel
Table SPARK_PACKAGES created successfully
Table USER_PREFERENCES created successfully
Table service_connections created successfully


In [4]:
# The code was removed by Watson Studio for sharing.

## Test Set

In [5]:
test_identity = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(cos.url('test_identity.csv', 'default-donotdelete-pr-l16zufkrkht8vm'))

In [6]:
test_transaction = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(cos.url('test_transaction.csv', 'default-donotdelete-pr-l16zufkrkht8vm'))

In [7]:
test_identity.createOrReplaceTempView("test_identity")
test_transaction.createOrReplaceTempView("test_transaction")

### Joining Transaction and Identity

In [8]:
columns = test_identity.columns[1:]
queryCols = ''
for col in columns:
    queryCols += ', b.'+col
queryCols

', b.id_01, b.id_02, b.id_03, b.id_04, b.id_05, b.id_06, b.id_07, b.id_08, b.id_09, b.id_10, b.id_11, b.id_12, b.id_13, b.id_14, b.id_15, b.id_16, b.id_17, b.id_18, b.id_19, b.id_20, b.id_21, b.id_22, b.id_23, b.id_24, b.id_25, b.id_26, b.id_27, b.id_28, b.id_29, b.id_30, b.id_31, b.id_32, b.id_33, b.id_34, b.id_35, b.id_36, b.id_37, b.id_38, b.DeviceType, b.DeviceInfo'

In [9]:
query = f"""
SELECT 
    a.* {queryCols} 
    FROM test_transaction a
    LEFT JOIN test_identity b
    ON a.transactionID = b.transactionID
"""

transaction_identity = sql(query)
transaction_identity.createOrReplaceTempView("transaction_identity")

In [10]:
test_identity.count()

141907

In [12]:
test_transaction.count()

506691

In [13]:
transaction_identity.count()

506691

In [ ]:
display(transaction_identity)

Training Set

In [11]:
# We have already ETLed the training set in data exploration
df_train = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(cos.url('transaction_id_train.csv', 'default-donotdelete-pr-l16zufkrkht8vm'))


In [12]:
df_train.count()

590540

In [13]:
# select only columns in both training and test
col_train = df_train.columns

In [14]:
col_query = col_train[0]
for col in col_train[1:]:
    if col != 'isFraud':
        col_query = col_query + ', ' + col
col_query

'D13, D14, D12, id_04, id_03, D6, D9, D8, id_09, id_10, id_32, id_14, V153, V161, V162, V163, V138, V157, V156, V155, V154, V139, V149, V158, V147, V146, V142, V141, V140, V148, V152, V151, V150, V145, V144, V143, V159, V160, V165, V164, V166, V322, V332, V339, V338, V337, V336, V335, V334, V333, V331, V323, V329, V328, V327, V326, V325, V324, V330, id_13, V231, V252, V244, V246, V247, V248, V249, V253, V242, V254, V257, V258, V218, V261, V262, V263, V243, V241, V265, V240, V237, V236, V235, V233, V232, V230, V229, V228, V226, V225, V224, V223, V217, V264, V260, V266, V273, V267, V278, V277, V276, V275, V274, V219, V268, V269, id_05, id_06, id_20, id_19, id_17, V177, V213, V179, V178, V176, V214, V173, V172, V168, V215, V216, V167, V212, V202, V181, V203, V211, V207, V183, V186, V187, V206, V191, V196, V205, V204, V193, V199, V192, V182, V190, V188, V174, V184, V189, V180, V194, V195, V197, V198, V200, V201, V175, V185, V170, V208, V209, V210, V169, V171, id_02, id_11, V220, V239, V234

In [15]:
query = f"""
SELECT 
    {col_query}
    FROM transaction_identity
"""

transaction_identity2 = sql(query)
transaction_identity2.createOrReplaceTempView("transaction_identity2")

In [16]:
# save
path = "cos://default-donotdelete-pr-l16zufkrkht8vm.os_938e241641654cf0a8c1bb0aefbea33d_configs/transaction_id_test.csv"
transaction_identity2.write.format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
    .option('header', 'true').save(path)

## Data Cleansing

In [16]:
df_test = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(cos.url('transaction_id_test.csv', 'default-donotdelete-pr-l16zufkrkht8vm'))

In [18]:
df_test.count()

506691

In [19]:
# df_train loaded already above
df_train.createOrReplaceTempView("df_train")
df_test.createOrReplaceTempView("df_test")

### Data Types

In [ ]:
display(df_train)

In [21]:
display(df_test)

In [22]:
# get categorical and numerical columns
cat = ['id_33', 'id_30', 'id_34', 'DeviceInfo', 'id_16', 
       'P_emaildomain', 'R_emaildomain', 'id_31', 'DeviceType', 
       'id_29', 'id_28', 'id_38', 'id_35', 'id_37', 'id_15',
      'id_36', 'card4', 'card6', 'ProductCD', 'id_12'] 
num = [c for c in df_train.columns if c not in cat]

numcat = num + cat
types = ['float' for i in range(len(num))] + ['string' for i in range(len(cat))]
numcat_types = list(zip(numcat, types))
print(numcat_types[:5])
print(numcat_types[-5:])

[('D13', 'float'), ('D14', 'float'), ('D12', 'float'), ('id_04', 'float'), ('id_03', 'float')]
[('id_36', 'string'), ('card4', 'string'), ('card6', 'string'), ('ProductCD', 'string'), ('id_12', 'string')]


In [ ]:
# converting numbers from string to float

df_train2 = df_train.select(*(df_train[col].cast(types).alias(col)
                        for col, types in numcat_types))
display(df_train2)

In [ ]:
# converting numbers from string to float

# get categorical and numerical columns
cat = ['id_33', 'id_30', 'id_34', 'DeviceInfo', 'id_16', 
       'P_emaildomain', 'R_emaildomain', 'id_31', 'DeviceType', 
       'id_29', 'id_28', 'id_38', 'id_35', 'id_37', 'id_15',
      'id_36', 'card4', 'card6', 'ProductCD', 'id_12'] 
num = [c for c in df_test.columns if c not in cat]

numcat = num + cat
types = ['float' for i in range(len(num))] + ['string' for i in range(len(cat))]
numcat_types = list(zip(numcat, types))
print(numcat_types[:5])
print(numcat_types[-5:])

df_test2 = df_test.select(*(df_test[col].cast(types).alias(col)
                        for col, types in numcat_types))
display(df_test2)

In [25]:
df_train2.createOrReplaceTempView('df_train2')
df_test2.createOrReplaceTempView('df_test2')

### Duplicates

In [26]:
query = """
SELECT COUNT(*), COUNT(DISTINCT(transactionID))
FROM df_train2
"""

sql(query).show()

+--------+-----------------------------+
|count(1)|count(DISTINCT transactionID)|
+--------+-----------------------------+
|  590540|                       590540|
+--------+-----------------------------+



In [27]:
query = """
SELECT COUNT(*), COUNT(DISTINCT(transactionID))
FROM df_test2
"""

sql(query).show()

+--------+-----------------------------+
|count(1)|count(DISTINCT transactionID)|
+--------+-----------------------------+
|  506691|                       506691|
+--------+-----------------------------+



### Null Values

In [28]:
from pyspark.sql.functions import col, sum

# replace all null values with 0
df_train3 = df_train2.fillna(value=0)

nulls = df_train3.select(*(sum(col(c).isNull().cast("int")).alias(c) for c in df_train3.columns))
nulls_pd = nulls.toPandas().transpose()

num_trans = df_train3.count()
nulls_pd.columns = ['counts']
nulls_pd["prop"] = nulls_pd.counts / num_trans
nulls_pd.sort_values(by="prop", ascending=False)

,counts,prop
id_33,517251,0.875895
id_30,512975,0.868654
id_34,512735,0.868248
DeviceInfo,471874,0.799055
id_16,461200,0.780980
R_emaildomain,453249,0.767516
id_31,450258,0.762451
DeviceType,449730,0.761557
id_28,449562,0.761273
id_29,449562,0.761273


In [29]:
# replace all null values with 0
df_test3 = df_test2.fillna(value=0)

nulls = df_test3.select(*(sum(col(c).isNull().cast("int")).alias(c) for c in df_test3.columns))
nulls_pd = nulls.toPandas().transpose()

num_trans = df_test3.count()
nulls_pd.columns = ['counts']
nulls_pd["prop"] = nulls_pd.counts / num_trans
nulls_pd.sort_values(by="prop", ascending=False)

,counts,prop
id_30,436032,0.860548
id_33,436020,0.860524
id_34,434516,0.857556
DeviceInfo,391634,0.772925
id_16,380944,0.751827
R_emaildomain,370821,0.731848
id_31,370066,0.730358
id_28,369913,0.730056
id_29,369913,0.730056
DeviceType,369760,0.729754


In [30]:
# save test
# save
path = "cos://default-donotdelete-pr-l16zufkrkht8vm.os_938e241641654cf0a8c1bb0aefbea33d_configs/transaction_id_test_nonull.csv"
df_test3.write.format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
    .option('header', 'true').save(path)

In [35]:
# save train
# save
path = "cos://default-donotdelete-pr-l16zufkrkht8vm.os_938e241641654cf0a8c1bb0aefbea33d_configs/transaction_id_train_nonull.csv"
df_train3.write.format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
    .option('header', 'true').save(path)

### Stats Per Numerical Column

In [ ]:
# min, max, mean, stddev

stats_train = df_train3.describe()
display(stats_train)

In [ ]:
# min, max, mean, stddev

stats_test = df_test3.describe()
display(stats_test)

In [33]:
df_test3.count(), df_train3.count()

(506691, 590540)